In [ ]:
import json

from sqlalchemy import create_engine, text
import pandas as pd

# 数据库配置
import pickle
with open('./datasets/sql_token.pkl', 'rb') as f:
    DB_URI = pickle.load(f)

engine = create_engine(DB_URI)
DATA_JSON_PATH = './datasets/data.json'

# 加载分类数据
with open(DATA_JSON_PATH) as f:
    group_map = json.load(f)


In [ ]:
for group_idx, (group_name, group_list) in enumerate(group_map.items()):
    for group_data in group_list:
        if group_name == 'stock1' and group_data['industry_code'] in ['270000', '480000']:
            print(group_name, group_data, )
        # break
    # break

In [ ]:
for i in range(len(group_map['stock1'])):
    if group_map['stock1'][i]['industry_code'] == '270000':
        print(group_map['stock1'][i])

for i in range(len(group_map['stock1'])):
    if group_map['stock1'][i]['industry_code'] == '480000':
        print(group_map['stock1'][i])

In [ ]:
pred_value = 'accnav'
sql = text(f"SELECT fund_code, date, nav, accnav, adj_nav FROM b_fund_nav_details_new WHERE fund_code IN :codes AND date BETWEEN :start AND :end ORDER BY date")
df = pd.read_sql_query(
    sql.bindparams(codes=tuple(['013152']), start='2020-07-13', end='2025-03-8'),
    engine
)
df

In [ ]:
df_stamp = pd.to_datetime(df['date'])
df_stamp[1].year, df_stamp[1].month, df_stamp[1].day, df_stamp[1].weekday()

In [ ]:
from c_model_center import Model
from a_data_center import DataModule
# Experiment Settings, logger, plotter
from utils.exp_config import get_config
config = get_config()

datamodule = DataModule(config)
model = Model(datamodule, config)
datamodule.test_x[0]

In [ ]:
datamodule.valid_x[0]

In [ ]:
datamodule.train_x[0]

In [ ]:
9 * 160 + 200 * 7

In [ ]:
a = torch.randn(2, 2)

In [ ]:
import torch
# a = torch.tensor([1, 1, 12, 9, 8, 2, 11], dtype=torch.float32).unsqueeze(1)  # shape (6, 1)
a /= a.max()
# cls = torch.randn(a.shape[-1]).unsqueeze(-1)
cls = torch.randn(2, 2)
a = torch.cat((cls, a), dim=0)
dk = a.shape[-1]
qk = torch.matmul(a, a.T) / dk  # shape (6, 6)
weight = torch.softmax(qk, dim=-1)
out = torch.matmul(weight, a)
print(a)

In [ ]:
print(qk)

In [ ]:
print(weight)

In [ ]:

print(out)

In [ ]:
import pandas as pd
import numpy as np
df = pd.read_excel('./datasets/lottery/pl3_desc.xls', header=1)
timestamps = pd.to_datetime(df['开奖日期'])
timestamps = np.array([[ts.year, ts.month, ts.day, ts.weekday()] for ts in timestamps])
x = np.array(df['号']).reshape(-1, 1)
# timestamps
x = np.concatenate((timestamps, x), axis=-1)
df

In [ ]:
from modules.pretrain_timer import Timer
import torch
from utils.exp_config import get_config
config = get_config()
config.patch_len = 96
config.d_model = 1024
config.d_ff = 2048
config.e_layers = 8
config.n_heads = 8
config.dropout = 0.10
config.factor = 1
config.output_attention = 1
config.activation = 'gelu'
config.ckpt_path = 'Timer_forecast_1.0.ckpt'
backbone = Timer(config)
ckpt_path = 'Timer_forecast_1.0.ckpt'
sd = torch.load(ckpt_path, weights_only=False, map_location="cpu")["state_dict"]
sd = {k[6:]: v for k, v in sd.items()}
backbone.load_state_dict(sd, strict=True)